# TF-IDF
- 각 문서별(행)별로 자주 나타나는 단어들에 가중치를 두면서 텍스트를 벡터로 변환시켜주는 과정이다 . 

# 코사인 유사도
- 위와 같은 다양한 텍스트마이닝 기법으로 텍스트를 벡터화 시킨 후 , 벡터간 $cos(\theta)$를 구해서 , 벡터간 유사성을 알 수 있다.

참고링크 : https://wikidocs.net/24603

데이터셋 : https://www.kaggle.com/rounakbanik/the-movies-dataset

>- 캐글에서 사용되었던 영화 데이터셋을 가지고 영화 추천 시스템 구현

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('C:/Users/병현/Desktop/kaggle/movies_metadata.csv/movies_metadata.csv', low_memory = False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
print(data.shape)
# 데이터는 총 45466개이지만 , 20000개만 사용하기로 함

data = data.head(20000)

(45466, 24)


In [4]:
data['overview'].isnull().sum()

135

In [5]:
data['overview'] = data['overview'].fillna('')
# overview에서 Null 값을 가진 경우에는 값 제거

In [6]:
data[['title', 'overview']]

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
19995,Rebellion,Dissidents in a French colony attack a police ...
19996,Versailles,A young mother Nina and her son Enzo find them...
19997,Two in the Wave,An in-depth analysis of the relationship betwe...
19998,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...


# TfidfVectorizer 사용

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(20000, 47487)


>- 총 47487개의 단어가 사용되었음
>- tfidf_matrix는 20000 X 47487 행렬

In [8]:
# 코사인 유사도를 구하는 과정
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
# 영화의 타이틀을 입력하면 인덱스를 리턴하기 위해서 생성

indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [10]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

# 모델 평가하기 

In [12]:
get_recommendations('The Dark Knight Rises')




12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

>- 다크나이트와 유사한 영화를 추천받고자 햇는데 , 대부분 배트맨과 관련된 영화를 출력하는것으로써 매우 잘 됐음을 볼 수 있다.